# Image Analysis with AI Models

This notebook demonstrates how to use Snowflake Cortex to analyze images with AI models like Claude 3.5 Sonnet and Pixtral-large.

## Setup

First, let's import the required packages and set up our session:

In [ ]:
# Import required packages
import pandas as pd
import json
from snowflake.snowpark.context import get_active_session


# Setup session
session = get_active_session()
session.use_schema("image_analysis")

## View Available Images

Let's see what images we have in our table:

In [ ]:
# View available images
image_df = session.sql("SELECT * FROM image_table").collect()
print(f"Found {len(image_df)} images in the stage")

# Display the first few images
for i, row in enumerate(image_df[:3]):
    print(f"Image {i+1}: {row['IMAGE_PATH']}")

## Image Analysis Function

Create a function to analyze images with our AI models:

In [ ]:
# Function to analyze an image with chosen AI model
def analyze_image(image_path, prompt_template, model='claude-4-sonnet'):
    # Escape single quotes in the prompt
    escaped_prompt = prompt_template.replace("'", "''")
    
    sql_query = f"""
    SELECT 
      SNOWFLAKE.CORTEX.COMPLETE(
        '{model}', 
        '{escaped_prompt}', 
        TO_FILE('@images', '{image_path}')
      )
    """
    result = session.sql(sql_query).collect()
    return result[0][0]

## Basic Image Analysis

Let's analyze our first image with a custom prompt:

In [ ]:
# Example analysis with custom prompt
if len(image_df) > 0:
    image_path = image_df[0]['IMAGE_PATH']  # Get first image
    prompt = "Describe whats happening in this image in the style of a detective investigating a scene. Be brief but detailed."

    analysis = analyze_image(image_path, prompt)
    print(f"Analysis for {image_path}:")
    print(analysis)
else:
    print("No images found in the table.")

## Object Detection

Now let's identify objects in the image:

In [ ]:
# Object detection
if len(image_df) > 0:
    image_path = image_df[0]['IMAGE_PATH']
    object_prompt = "List all visible objects in this image and count how many there are of each type."
    object_analysis = analyze_image(image_path, object_prompt)
    print("Objects detected:")
    print(object_analysis)
else:
    print("No images found in the table.")

## Text Extraction

Let's extract any visible text from the image:

In [ ]:
# Text extraction
if len(image_df) > 0:
    image_path = 'car_ad_1.PNG'
    text_prompt = "Extract and transcribe any visible text in this image. If no text is visible, respond with No text detected."
    text_analysis = analyze_image(image_path, text_prompt)
    print("Text extracted:")
    print(text_analysis)
else:
    print("No images found in the table.")

## Try Different Model

Let's compare with the Pixtral-large model:

In [ ]:
# Try with Pixtral model
if len(image_df) > 0:
    image_path = image_df[0]['IMAGE_PATH']
    prompt = "Describe what you see in this image. Be detailed but concise."
    pixtral_analysis = analyze_image(image_path, prompt, model='pixtral-large')
    print(f"Pixtral analysis for {image_path}:")
    print(pixtral_analysis)
else:
    print("No images found in the table.")

## Image Comparison

Let's compare two images and get structured output:

In [ ]:
# Compare two images and get structured output
def compare_images(image_path1, image_path2, model='claude-4-sonnet'):
    sql_query = f"""
    SELECT snowflake.cortex.complete('{model}', 
    Prompt('Compare this image {{0}} to this  image {{1}}. Respond to the following questions in a JSON Object. Background Color: Do the images use a similar or different background color? Color: Do the images use similar or different colors? Mannequin Count: Do the displays have the same number of Mannequins? For each question answer yes or no and a brief 5 word description', 
    TO_FILE('@images', '{image_path1}'), 
    TO_FILE('@images', '{image_path2}')
    ))
    """
    
    result = session.sql(sql_query).collect()
    
    return result[0][0]

# Get two images to compare
if len(image_df) >= 2:
    image1 = 'car_ad_1.PNG'
    image2 = 'car_ad_2.PNG'
    
    print(f"Comparing {image1} and {image2}:\n")
    comparison_result = compare_images(image1, image2)
    print(comparison_result)
else:
    print("Need at least two images for comparison")

## Batch Processing

Let's create a system for batch processing multiple images:

In [ ]:

-- Create image table from stage
CREATE OR REPLACE TABLE image_table AS 
(SELECT to_file('@images', RELATIVE_PATH) as img from directory(@images));

-- Process all images with basic classification
SELECT 
    SNOWFLAKE.CORTEX.COMPLETE('claude-4-sonnet',
    PROMPT('Classify the input image {0} in no more than 2 words. Respond in JSON', img)) as image_classification
FROM image_table;